In [2]:
import sys
import os
import nest_asyncio
import asyncpg
from pyspark.sql import SparkSession
import pandas as pd

In [3]:

nest_asyncio.apply()

DB_CONFIG = {
    "user": os.getenv("POSTGRES_USER", "admin"),
    "password": os.getenv("POSTGRES_PASSWORD", "admin"),
    "database": os.getenv("POSTGRES_DB", "tender"),
    "host": os.getenv("POSTGRES_HOST", "postgre"),
    "port": int(os.getenv("POSTGRES_PORT", 5432))
}

async def read_table_as_df() -> pd.DataFrame:
    conn = await asyncpg.connect(**DB_CONFIG)
    rows = await conn.fetch("SELECT * FROM xml_storage")
    df = pd.DataFrame([dict(row) for row in rows])
    await conn.close()
    return df

df = await read_table_as_df()

if df.empty:
    print("Dataframe is empty, exiting program.")
    sys.exit()

gaierror: [Errno 11001] getaddrinfo failed

In [ ]:
spark = SparkSession.builder \
                    .appName("File_to_spark") \
                    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

In [ ]:
df = spark.createDataFrame(df)

NameError: name 'spark' is not defined